In [1]:
import polars as pl
import polars_st as st
import hvplot.polars
import json
from palettable.cartocolors.qualitative import Bold_9
import re
import requests

import polars_h3 as plh3
from datetime import date


In [2]:
# # only need to run once
# precincts = json.load(open("data/precincts.json"))

# for p in precincts:
#     res = requests.get(
#         f"https://geosearch.planninglabs.nyc/v2/search?text={p['address']}&size=1"
#     ).json()

#     if res["features"]:
#         geom = res["features"][0]["geometry"]
#         p["geometry"] = geom
#     else:
#         p["geometry"] = None

# df_precincts = pl.from_dicts(precincts).with_columns(
#     geometry=st.from_geojson(pl.col("geometry").struct.json_encode()).st.set_srid(4326)
# )
# df_precincts.write_parquet("data/precincts.parquet")

In [3]:
# # only run once to create combined parquet
# df_speeds1 = pl.scan_csv(
#     "data/MTA_Bus_Route_Segment_Speeds__2023_-_2024_20250921.csv",
# )
# df_speeds2 = pl.scan_csv(
#     "data/MTA_Bus_Route_Segment_Speeds__Beginning_2025_20250918.csv",
# )
# df_speeds = (
#     pl.concat([df_speeds1, df_speeds2])
#     .with_columns(
#         pl.col("Timestamp")
#         .str.strptime(pl.Datetime, "%m/%d/%Y %I:%M:%S %p")
#         .name.keep(),
#         timepoint_geom=st.from_wkt("Timepoint Stop Georeference").st.set_srid(4326),
#         next_timepoint_geom=st.from_wkt("Next Timepoint Stop Georeference").st.set_srid(
#             4326
#         ),
#     )
#     .drop(
#         "Timepoint Stop Latitude",
#         "Timepoint Stop Longitude",
#         "Timepoint Stop Georeference",
#         "Next Timepoint Stop Latitude",
#         "Next Timepoint Stop Longitude",
#         "Next Timepoint Stop Georeference",
#     )
# )
# df_speeds.collect().write_parquet("data/speeds.parquet")

In [4]:
df = pl.read_parquet("data/data.parquet")
# df_speeds = pl.read_parquet("data/speeds.parquet")
df_ace = pl.read_csv(
    "data/MTA_Bus_Automated_Camera_Enforced_Routes__Beginning_October_2019_20250921.csv"
).rename(lambda col: col.strip().lower().replace(" ", "_"))
# From: "https://trainstat.us/api/v1/routes?route_type=bus&geojson=true"
df_routes = st.read_file("data/routes.geojson")
# https://trainstat.us/api/v1/stops?route_type=bus&geojson=true
# gdf_cuny = gpd.read_file("https://data.ny.gov/resource/irqs-74ez.geojson")
df_cuny = st.read_file("data/cuny_campuses.geojson", columns=["campus", "address"])
# https://trainstat.us/api/v1/stops?route_type=bus&geojson=false
df_stops = pl.read_json("data/stops.json").with_columns(
    pl.col("routes").list.eval(pl.element().struct.field("id")).name.keep(),
    geometry=st.point(pl.concat_arr("lon", "lat")).st.set_srid(4326),
)
# srxy-5nxn
df_cbd = st.read_file("data/cbd.geojson")
df_precincts = pl.read_parquet("data/precincts.parquet")

In [5]:
df_speeds1 = pl.read_csv("data/MTA_Bus_Speeds__2015-2019_20250924.csv")
df_speeds2 = pl.read_csv("data/MTA_Bus_Speeds__2020_-_2024_20250924.csv")
df_speeds3 = pl.read_csv("data/MTA_Bus_Speeds__Beginning_2025_20250924.csv")
df_speeds = pl.concat(
    [df_speeds1, df_speeds2, df_speeds3], how="diagonal_relaxed"
).with_columns(pl.col("month").str.to_date())

In [6]:
df = df.with_columns(
    pl.col("First Occurrence")
    .str.strptime(pl.Datetime, "%m/%d/%Y %I:%M:%S %p")
    .name.keep(),
    pl.col("Last Occurrence")
    .str.strptime(pl.Datetime, "%m/%d/%Y %I:%M:%S %p")
    .name.keep(),
    # from looking at the points, it seems like M14+ should be M14A+ (M14B+ doesn't have ACE)
    pl.col("Bus Route ID").str.replace("M14+", "M14A+", literal=True).name.keep(),
    geometry=st.from_wkt("Violation Georeference").st.set_srid(4326),
    stop_geometry=st.from_wkt("Bus Stop Georeference").st.set_srid(4326),
).drop(
    "Violation Longitude",
    "Violation Latitude",
    "Violation Georeference",
    "Bus Stop Georeference",
    "Bus Stop Latitude",
    "Bus Stop Longitude",
)

In [7]:
df_ace = df_ace.with_columns(
    pl.col("implementation_date").str.strptime(pl.Date, "%m/%d/%Y").name.keep()
)

In [8]:
df_precincts.select(st.geom(), "name").st.write_file("parsed/precincts.geojson")

In [9]:
df_nyc_bounds = st.read_file(
    requests.get("https://data.cityofnewyork.us/resource/gthc-hcne.geojson").text
).select(st.geom().st.union_all())

In [10]:
df_usps = st.read_file("data/usps_nyc.geojson").filter(pl.col("Status") == "OPEN")
df_usps = df_usps.st.sjoin(df_nyc_bounds).drop("geometry_right")
df_usps.select(st.geom(), "OBJECTID").st.write_file("parsed/usps.geojson")

In [11]:
df_health_facilities = pl.read_csv(
    "data/Health_Facility_General_Information_20250924.csv"
)
df_health_facilities = (
    df_health_facilities.with_columns(
        geometry=st.point(
            pl.concat_arr("Facility Longitude", "Facility Latitude")
        ).st.set_srid(4326)
    )
    .filter(
        pl.col("Facility Longitude").is_not_null()
        & pl.col("Facility Latitude").is_not_null()
    )
    .st.sjoin(df_nyc_bounds)
    .drop("geometry_right")
)
df_health_facilities.select("Facility ID", "Facility Name", st.geom()).st.write_file(
    "parsed/health_facilities.geojson"
)

In [12]:
# df_nearby_routes = (
#     df_stops.with_columns(st.geom().st.to_srid(2263))
#     .st.sjoin(
#         df_cuny.with_columns(st.geom().st.to_srid(2263)),
#         predicate="dwithin",
#         distance=500,
#     )
#     .select(pl.col("routes").alias("id"), "campus")
#     .explode("id")
#     .unique("id")
# )
# (
#     df_stops.with_columns(st.geom().st.to_srid(2263))
#     .st.sjoin(
#         df_cuny.with_columns(st.geom().st.to_srid(2263)),
#         predicate="dwithin",
#         distance=500,
#     )
#     .select(pl.col("routes").alias("id"), "campus")
#     .explode("id")
#     .group_by("id")
#     .agg(pl.col("campus"))
#     # .pivot(
#     #     index="id", on="campus", values="campus"
#     # )
#     # .unique("id")
# )
# Get all unique campus names first
campus_names = df_cuny["campus"].unique().to_list()

# Create the base nearby routes with all campuses per route
df_nearby_routes_full = (
    df_stops.with_columns(st.geom().st.to_srid(2263))
    .st.sjoin(
        df_cuny.with_columns(st.geom().st.to_srid(2263)),
        predicate="dwithin",
        distance=500,
    )
    .select(pl.col("routes").alias("id"), "campus")
    .explode("id")
    .unique(["id", "campus"])  # Remove any duplicates
)

# Create boolean columns for each campus
# Start with all route IDs from stops
all_route_ids = (
    df_stops.select("routes").explode("routes").unique().rename({"routes": "id"})
)

# For each campus, create a boolean indicator
campus_boolean_dfs = []
campus_mapping = []
for campus in campus_names:
    campus_safe = campus.lower().replace(" ", "_").replace("-", "_")
    campus_routes = df_nearby_routes_full.filter(pl.col("campus") == campus)[
        "id"
    ].to_list()

    campus_bool_df = all_route_ids.with_columns(
        pl.col("id").is_in(campus_routes).alias(campus_safe)
    ).select("id", campus_safe)

    campus_boolean_dfs.append(campus_bool_df)
    campus_mapping.append({"original": campus, "sanitized": campus_safe})

# Join all campus boolean columns together
df_nearby_routes_boolean = all_route_ids.select("id")
for campus_df in campus_boolean_dfs:
    df_nearby_routes_boolean = df_nearby_routes_boolean.join(
        campus_df, on="id", how="left"
    )

# Fill any nulls with False (routes not near any campus)
df_nearby_routes_boolean = df_nearby_routes_boolean.fill_null(False)

# Now join this with your routes dataframe
df_routes = df_routes.join(df_nearby_routes_boolean, on="id", how="left").join(
    df_ace.unique("route").select(pl.col("route").alias("id"), "implementation_date"),
    on="id",
    how="left",
)

In [13]:
with open("src/lib/assets/campus_mapping.json", "w") as f:
    json.dump(campus_mapping, f, indent=2)

campus_list = [campus for campus in campus_names]
with open("src/lib/assets/campus_list.json", "w") as f:
    json.dump(campus_list, f, indent=2)

In [14]:
# non M routes will prob go far outside of CBD, so just focus on M routes
routes_in_cbd = (
    df_routes.st.sjoin(
        df_cbd,
    )
    .filter(pl.col("id").str.contains("M"))["id"]
    .unique()
    .to_list()
)

# (
#     df_routes.st.sjoin(
#         df_cbd,
#     )
#     .filter(pl.col("id").str.contains("M"))
#     .st.explore()
# )
# .select(pl.col("id").alias("cbd_id")).unique()
# df_routes.with_columns(
#     in_cbd=st.geom()
#     .st.to_srid(2263)
#     .st.intersects(df_cbd.with_columns(st.geom().st.to_srid(2263)).st.union_all())
# )
df_routes = df_routes.with_columns(in_cbd=pl.col("id").is_in(routes_in_cbd))

In [15]:
df_routes.with_columns(pl.col("implementation_date").dt.epoch("s")).st.write_file(
    "parsed/bus_routes.geojson"
)

df_cuny.st.write_file("parsed/cuny_campuses.geojson")

In [16]:
# df_cuny["campus"].to_list()
print(df_ace["implementation_date"].min(), df_ace["implementation_date"].max())

2019-10-07 2025-09-15


In [17]:
df_ace_routes = df_ace.select(["route", "implementation_date"]).unique("route")

df_speeds_ace = (
    df_speeds.join(df_ace_routes, left_on="route_id", right_on="route", how="left")
    # .drop("route")
    .with_columns(
        is_ace=pl.col("implementation_date").is_not_null(),
        ace_phase=pl.when(
            pl.col("implementation_date").is_not_null()
            & (pl.col("month") >= pl.col("implementation_date"))
        )
        .then(pl.lit("Post-ACE"))
        .when(
            pl.col("implementation_date").is_not_null()
            & (pl.col("month") < pl.col("implementation_date"))
        )
        .then(pl.lit("Pre-ACE"))
        .otherwise(pl.lit("No ACE")),
    )
)

# OPTIONAL filters (uncomment as needed)
df_speeds_ace = df_speeds_ace.filter(pl.col("period") == "Peak")
# df_speeds_ace = df_speeds_ace.filter(pl.col("day_type") == 1)  # e.g., weekdays if 1=weekday

# Aggregate weighted average speeds pre vs post (ACE routes only)
agg = (
    df_speeds_ace.filter(pl.col("is_ace"))
    .group_by(["route_id", "ace_phase"])
    .agg(
        weighted_avg_speed=(
            (pl.col("average_speed") * pl.col("total_operating_time")).sum()
            / pl.col("total_operating_time").sum()
        ),
        months=pl.len(),
        total_time=pl.col("total_operating_time").sum(),
        total_miles=pl.col("total_mileage").sum(),
    )
)

# Keep routes that have both Pre-ACE and Post-ACE data
agg_both = agg.filter(pl.col("ace_phase").n_unique().over("route_id") == 2)

# Pivot to wide and compute deltas
pre_post_wide = (
    agg_both.pivot(
        index=["route_id"],
        on="ace_phase",
        values="weighted_avg_speed",
        aggregate_function="first",
    )
    .with_columns(
        delta_mph=pl.col("Post-ACE") - pl.col("Pre-ACE"),
        pct_change=pl.when(pl.col("Pre-ACE") != 0)
        .then(((pl.col("Post-ACE") - pl.col("Pre-ACE")) / pl.col("Pre-ACE")) * 100)
        .otherwise(None),
    )
    .sort("delta_mph", descending=True)
)


In [18]:
# Quick visuals
# 1) Difference bar chart (top 20)
pre_post_wide.head(20).hvplot.bar(
    x="route_id",
    y="delta_mph",
    title="Speed Change After ACE (Top 20)",
    ylabel="Δ speed (mph)",
    rot=45,
)

# 2) Pre vs Post grouped bars for top 15 by absolute delta
top_routes = (
    pre_post_wide.with_columns(abs_delta=pl.col("delta_mph").abs())
    .sort("abs_delta", descending=True)["route_id"]
    .head(15)
    .to_list()
)
agg_both.filter(pl.col("route_id").is_in(top_routes)).hvplot.bar(
    x="route_id",
    y="weighted_avg_speed",
    by="ace_phase",
    stacked=False,
    title="Pre vs Post ACE (Top 15 by |Δ|)",
    ylabel="Weighted Avg Speed (mph)",
    rot=45,
)

:Bars   [route_id,ace_phase]   (weighted_avg_speed)

In [19]:
route_monthly = (
    df_speeds_ace
    # .filter(pl.col("is_ace"))  # uncomment to focus on ACE routes only
    # .filter(pl.col("period") == "Peak")  # optional
    # .filter(pl.col("day_type") == 1)     # optional weekdays
    .group_by(["route_id", "month"])
    .agg(
        weighted_avg_speed=(
            (pl.col("average_speed") * pl.col("total_operating_time")).sum()
            / pl.col("total_operating_time").sum()
        ),
        obs=pl.len(),
    )
    .sort(["route_id", "month"])
    .with_columns(
        rolling_3m=pl.col("weighted_avg_speed")
        .rolling_mean(window_size=3)
        .over("route_id")
    )
)

# 1) Interactive selector (dropdown) to pick a route
route_monthly.hvplot.line(
    x="month",
    y="weighted_avg_speed",
    groupby="route_id",
    title="Monthly Weighted Avg Speed by Route",
    ylabel="Speed (mph)",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'09fb2348-b382-4074-9426-910bc64a994d': {'version…

In [20]:
# LayerCake-ready time series (per-route monthly speeds)
route_monthly_export = route_monthly.select(
    pl.col("route_id").cast(pl.Utf8),
    pl.col("month").dt.strftime("%Y-%m-%d").alias("date"),
    pl.col("month").dt.epoch("ms").alias("ts"),  # epoch ms for easy Date parsing
    pl.col("weighted_avg_speed").alias("speed"),
    pl.col("rolling_3m").alias("speed_3m"),
)
# .with_columns(pl.col("speed_3m").fill_null(None))

# 1) Long format (array of rows)
route_monthly_export.write_json("src/lib/assets/route_speeds_long.json")

# 2) Nested per-series (grouped by route_id)
route_monthly_nested = route_monthly_export.group_by("route_id").agg(
    values=pl.struct(["ts", "date", "speed", "speed_3m"]).alias("values")
)
route_monthly_nested.write_json("src/lib/assets/route_speeds_nested.json")

# 3) ACE dates for vlines (optional)
df_ace_routes.select(
    pl.col("route").alias("route_id"),
    pl.col("implementation_date").dt.strftime("%Y-%m-%d").alias("date"),
    pl.col("implementation_date").dt.epoch("ms").alias("ts"),
).write_json("src/lib/assets/ace_dates.json")

# 4) Pre/Post summary for bars (rename keys for JS)
pre_post_wide_export = pre_post_wide.rename(
    {"Pre-ACE": "pre_ace", "Post-ACE": "post_ace"}
).select(["route_id", "pre_ace", "post_ace", "delta_mph", "pct_change"])
pre_post_wide_export.write_json("src/lib/assets/pre_post_summary.json")

# Question 2:

Some vehicles stopped in violation are exempt from fines due to business reasons. For vehicles that are exempt, are there repeat offenders? Where are exempt vehicles frequently in violation?


In [21]:
df_exempt = df.filter(pl.col("Violation Status") != "VIOLATION ISSUED")

In [22]:
df_exempt.height / df.height

0.3878956260678649

In [23]:
df["Violation Status"].value_counts(sort=True).with_columns(
    (pl.col("count") / pl.col("count").sum() * 100).alias("percentage")
).write_json("src/lib/assets/violation_status_summary.json")

df["Violation Status"].value_counts(sort=True).with_columns(
    (pl.col("count") / pl.col("count").sum() * 100).alias("percentage")
)

Violation Status,count,percentage
str,u32,f64
"""VIOLATION ISSUED""",2312878,61.210437
"""TECHNICAL ISSUE/OTHER""",320912,8.492953
"""EXEMPT - EMERGENCY VEHICLE""",286253,7.575701
"""DRIVER/VEHICLE INFO MISSING""",273968,7.250577
"""EXEMPT - COMMERCIAL UNDER 20""",257374,6.811416
"""EXEMPT - BUS/PARATRANSIT""",190192,5.033441
"""EXEMPT - OTHER""",136991,3.625474


Around 40% of all ACE violations were determined to be invalid.


In [24]:
df_exempt_grouped = (
    df_exempt.with_columns(coords=st.geom().st.coordinates().list.first())
    .with_columns(
        cell=plh3.latlng_to_cell(
            pl.col("coords").list.get(1), pl.col("coords").list.get(0), 12
        )
    )
    .group_by(["cell", "Violation Status"])
    .agg(
        [
            pl.len().alias("violation_count"),
            # pl.col("Violation Status").first(),
            pl.col("Violation Type").value_counts(),
            pl.col("Vehicle ID").n_unique().alias("unique_vehicles"),
            pl.col("Stop Name").first(),
            pl.col("Bus Route ID").first(),
            # (pl.col("First Occurrence").min()).alias("first_occurrence"),
            # (pl.col("Last Occurrence").max()).alias("last_occurrence"),
        ]
    )
    .with_columns(
        geometry=st.point(plh3.cell_to_latlng("cell").list.reverse(), srid=4326)
    )
    # .sort("violation_count", descending=True)
)

In [25]:
# # Alternative approach using explode + pivot
# df_exempt_grouped_pivoted = (
#     df_exempt.with_columns(coords=st.geom().st.coordinates().list.first())
#     .with_columns(
#         cell=plh3.latlng_to_cell(
#             pl.col("coords").list.get(1), pl.col("coords").list.get(0), 12
#         )
#     )
#     .group_by(["cell", "Violation Status", "Violation Type"])
#     .agg(
#         [
#             pl.len().alias("violation_type_count"),
#             pl.col("Vehicle ID").n_unique().alias("unique_vehicles_for_type"),
#             pl.col("Stop Name").first(),
#             pl.col("Bus Route ID").first(),
#         ]
#     )
#     # Pivot to get violation types as columns
#     .pivot(
#         index=["cell", "Violation Status", "Stop Name", "Bus Route ID"],
#         on="Violation Type",
#         values="violation_type_count",
#         aggregate_function="sum",
#     )
#     .fill_null(0)  # Fill missing violation types with 0
#     # Add back other aggregations
#     .join(
#         df_exempt.with_columns(coords=st.geom().st.coordinates().list.first())
#         .with_columns(
#             cell=plh3.latlng_to_cell(
#                 pl.col("coords").list.get(1), pl.col("coords").list.get(0), 12
#             )
#         )
#         .group_by(["cell", "Violation Status"])
#         .agg(
#             [
#                 pl.len().alias("total_violation_count"),
#                 pl.col("Vehicle ID").n_unique().alias("unique_vehicles"),
#             ]
#         ),
#         on=["cell", "Violation Status"],
#     )
#     .with_columns(
#         geometry=st.point(plh3.cell_to_latlng("cell").list.reverse(), srid=4326)
#     )
# )

In [26]:
# df_exempt =
# df_violations_for_deckgl = (
#     df.rename(lambda col: col.lower().replace(" ", "_"))
#     .select(["violation_type", "violation_status", "geometry"])
#     # TODO: fix categorical ids not resetting to 0 for each col
#     # .with_columns_seq(
#     #     pl.selectors.string().cast(pl.Categorical).to_physical().name.suffix("_id"),
#     # )
#     .with_columns(
#         coords=st.geom().st.coordinates().list.first(),
#     )
#     .with_columns(
#         lng=pl.col("coords").list.get(0),
#         lat=pl.col("coords").list.get(1),
#     )
#     .drop("geometry", "coords")
#     # .select("violation_")
# )
df_violations_for_deckgl = (
    df_exempt.rename(lambda col: col.lower().replace(" ", "_"))
    .select(["violation_type", "violation_status", "geometry"])
    # TODO: fix categorical ids not resetting to 0 for each col
    # .with_columns_seq(
    #     pl.selectors.string().cast(pl.Categorical).to_physical().name.suffix("_id"),
    # )
    .with_columns(
        coords=st.geom().st.coordinates().list.first(),
    )
    .with_columns(
        lng=pl.col("coords").list.get(0),
        lat=pl.col("coords").list.get(1),
    )
    .drop("geometry", "coords")
    # .select("violation_")
)

In [27]:
status_col = "violation_status"

statuses = df_violations_for_deckgl.select(status_col).unique().to_series().to_list()

# build consistent color mapping
color_map = {}
for i, s in enumerate(sorted(statuses)):
    color_map[s] = Bold_9.colors[i % len(Bold_9.colors)]  # list of [r,g,b]

manifest = []


def _sanitize(name: str) -> str:
    # lower, replace any non-alphanumeric with underscore, collapse multiples, strip edges
    if name is None:
        return "unknown"
    out = re.sub(r"[^0-9a-zA-Z]+", "_", name.lower()).strip("_")
    return out or "unknown"


for status in statuses:
    safe = _sanitize(status)
    file_path = f"violations_status_{safe}.arrow"
    out_path = f"static/{file_path}"
    # filter and write IPC (keeps column types)
    df_status = df_violations_for_deckgl.filter(pl.col(status_col) == status)
    # if you only need numeric + lng/lat + small attrs, select them here to shrink file size
    # print(df_status.columns)
    df_status.select(pl.selectors.numeric()).write_ipc(
        out_path, compat_level=pl.CompatLevel.oldest()
    )
    manifest.append({"status": status, "file": file_path, "color": color_map[status]})

# write manifest + mapping for frontend
with open("src/lib/assets/violations_status_manifest.json", "w") as f:
    json.dump(manifest, f, indent=2)

with open("src/lib/assets/violations_status_color_map.json", "w") as f:
    json.dump({k: v for k, v in color_map.items()}, f, indent=2)

In [28]:
congestion_pricing_date = date(2025, 1, 5)

# ensure coords + month column exist
df_viol = df.with_columns(
    # coords=st.geom().st.coordinates().list.first(),
    # lng=pl.col("coords").list.get(0),
    # lat=pl.col("coords").list.get(1),
    month=pl.col("First Occurrence").dt.truncate("1mo"),
)

# spatial filter: approach 1 = CBD polygon, approach 2 = bbox
# if use_cbd:
#     df_cbd = df_viol.st.sjoin(cbd, predicate="intersects")
# else:
#     minx, miny, maxx, maxy = bbox
#     df_cbd = df_viol.filter(
#         (pl.col("lng") >= minx)
#         & (pl.col("lng") <= maxx)
#         & (pl.col("lat") >= miny)
#         & (pl.col("lat") <= maxy)
#     )

df_in_cbd = df_viol.st.sjoin(df_cbd, predicate="intersects")

# label pre / post congestion pricing
df_in_cbd = df_in_cbd.with_columns(
    cp_phase=pl.when(pl.col("First Occurrence").dt.date() < congestion_pricing_date)
    .then(pl.lit("pre"))
    .otherwise(pl.lit("post"))
)

# monthly counts by route and phase (for time series / charts)
counts = (
    df_in_cbd.group_by(["Bus Route ID", "month", "cp_phase"])
    .agg(pl.len().alias("violations"))
    .sort(["Bus Route ID", "month"])
)
# counts.write_json("src/lib/assets/violations_cbd_by_route.json")

summary = (
    counts.group_by(["Bus Route ID", "cp_phase"])
    .agg(pl.col("violations").sum().alias("violations"))
    .pivot(index="Bus Route ID", on="cp_phase", values="violations")
    .with_columns(
        pre=pl.col("pre").fill_null(0),
        post=pl.col("post").fill_null(0),
        # delta=pl.col("post") - pl.col("pre"),
        pct_change=pl.when(pl.col("pre") > 0)
        .then(((pl.col("post") - pl.col("pre")) / pl.col("pre")) * 100)
        .otherwise(None),
    )
)
# ensure JSON uses a simple route_id key (no spaces) for frontend lookups
summary = summary.rename({"Bus Route ID": "route_id"})
summary.write_json("src/lib/assets/violations_cbd_by_route_summary.json")

In [29]:
summary.join(df_ace_routes, left_on="route_id", right_on="route")

route_id,pre,post,pct_change,implementation_date
str,u32,u32,f64,date
"""M101""",17743,30525,72.039678,2024-09-16
"""M15+""",188073,62825,2.2836e6,2019-10-07
"""M34+""",14713,8902,2.9192e7,2020-08-10
"""M23+""",9284,6709,4.6262e7,2020-08-10
"""M42""",0,5644,null,2025-05-27
"""M2""",0,6868,null,2025-05-19
"""M4""",0,115,null,2025-05-19


In [30]:
strings_to_map = df_violations_for_deckgl.select(pl.selectors.string()).columns
# do it one at a time so categories always start at 0
for string_col in strings_to_map:
    df_violations_for_deckgl = df_violations_for_deckgl.with_columns(
        pl.col(string_col).cast(pl.Categorical).to_physical().name.suffix("_id"),
    )

    # get unique ids (integers) and build id->rgb mapping
    unique_ids = (
        df_violations_for_deckgl.select(f"{string_col}_id")
        .unique()
        .to_series()
        .to_list()
    )
    id_to_rgb = {}
    for i, uid in enumerate(sorted(unique_ids)):
        rgb = Bold_9.colors[i % len(Bold_9.colors)]
        id_to_rgb[uid] = rgb
    print(f"{string_col} id->rgb mapping:", id_to_rgb)
    # add an rgb column mapped from the id column (list of 3 ints)
    df_violations_for_deckgl = df_violations_for_deckgl.with_columns(
        pl.col(f"{string_col}_id").replace_strict(id_to_rgb).alias(f"{string_col}_rgb")
    )
    # display(df_violations_for_deckgl.select(string_col, f"{string_col}_id").unique())

violation_type id->rgb mapping: {0: [127, 60, 141], 1: [17, 165, 121], 2: [57, 105, 172]}
violation_status id->rgb mapping: {0: [127, 60, 141], 1: [17, 165, 121], 2: [57, 105, 172], 3: [242, 183, 1], 4: [231, 63, 116], 5: [128, 186, 90]}


In [31]:
# df["Violation Status"].value_counts()
color_mappings = {}

for string_col in strings_to_map:
    # display(
    #     df_violations_for_deckgl.select(
    #         string_col, f"{string_col}_id", f"{string_col}_rgb"
    #     )
    #     .unique()
    #     .sort(string_col)
    # )
    mapping = (
        df_violations_for_deckgl.select(
            string_col, f"{string_col}_id", f"{string_col}_rgb"
        )
        .unique()
        .sort(f"{string_col}_id")
        .write_json(f"src/lib/assets/{string_col}_mapping.json")
    )
    # color_mappings[string_col] =

In [32]:
df_violations_for_deckgl.select(pl.selectors.numeric()).write_ipc(
    "static/violations_by_status.arrow",
    # compression="zstd"
)

In [33]:
# df_violations_for_deckgl = (
#     df_exempt.rename(lambda col: col.lower().replace(" ", "_"))
#     .with_columns(
#         # violation_type_id=pl.col("violation_type").cast(pl.Categorical).to_physical(),
#         violation_status_id=pl.col("violation_status")
#         .cast(pl.Categorical)
#         .to_physical(),
#         coords=st.geom().st.coordinates().list.first(),
#     )
#     .with_columns(
#         lng=pl.col("coords").list.get(0),
#         lat=pl.col("coords").list.get(1),
#     )
#     # .with_columns(pl.selectors.string().cast(pl.Utf8).name.keep())
#     .select(
#         [
#             "lng",
#             "lat",
#             # "geometry",
#             "violation_status",
#             # "violation_type",
#             # pl.col("violation_type").cast(pl.Utf8),
#             # "vehicle_id",
#             # "bus_route_id",
#             # "stop_name",
#             # "violation_status",
#         ]
#     )
# )
# df_violations_for_deckgl.write_ipc(
#     "static/violations_by_status.arrow", compat_level=pl.CompatLevel.oldest()
# )
# # df_violations_for_deckgl.write_json("static/violations_by_status.json")
# # df_violations_for_deckgl.st.write_file(
# #     "static/violations_by_status.fgb", driver="FlatGeobuf"
# # )
# # df_violations_for_deckgl.write_csv("static/violations_by_status.csv")


In [33]:
# # BX28-BX38 are combined in ACE dataset
# union_geom = (
#     df_bus.filter(pl.col("id").is_in(["BX28", "BX38"]))
#     .select(st.geom().st.union_all())
#     .item()
# )

# other_cols = (
#     df_bus.filter(pl.col("id").is_in(["BX28", "BX38"]))
#     .select(
#         pl.exclude("geometry", "id"),
#     )
#     .head(1)
# )
# # union_geom

# # create new row with unioned geometry and other columns from one of the rows
# new_row = pl.DataFrame(
#     {
#         "id": ["BX28-BX38"],
#         "geometry": [union_geom],
#         # Add other columns that exist in df_bus with appropriate values
#         **other_cols.to_dict(),
#     }
# )

# df_bus = pl.concat([df_bus, new_row], how="diagonal")